<a href="https://colab.research.google.com/github/Vinayak-Sharma12/Web-Scraping/blob/main/WebScraping_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install beautifulsoup4

In [7]:
import faiss
import numpy as np
import requests
from bs4 import BeautifulSoup
#from sentence_transformers import SentenceTransformer

# Step 1: Scrape Website Content
def scrape_website(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = [p.get_text().strip() for p in soup.find_all("p")]
    return [p for p in paragraphs if p]

# Example Usage
url = "https://mrproptek.com/about"  # Updated with actual website
texts = scrape_website(url)

In [8]:
print(texts)

[]


In [ ]:
!pip install faiss-cpu numpy requests beautifulsoup4 sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import faiss
import numpy as np
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# ---------------------------
# Step 1: Scrape Website Content
# ---------------------------
def scrape_website(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = [p.get_text().strip() for p in soup.find_all("p")]
    return [p for p in paragraphs if p]

# ---------------------------
# Step 2: Embed Text using Sentence Transformers
# ---------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def embed_text(texts):
    if not texts:
        print("No text extracted from website.")
        return np.array([])
    return embedder.encode(texts, normalize_embeddings=True)

# ---------------------------
# Step 3: Build FAISS Index
# ---------------------------
def build_faiss_index(embeddings):
    if embeddings.size == 0:
        print("No embeddings available to add to FAISS index.")
        return None
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)  # Inner product for cosine similarity (with normalized embeddings)
    index.add(embeddings)
    return index

# ---------------------------
# Step 4: Search FAISS Index for Relevant Passages
# ---------------------------
def search_faiss(index, query, texts, top_k=3):
    if index is None:
        print("FAISS index is empty. Cannot perform search.")
        return []
    query_embedding = embedder.encode([query], normalize_embeddings=True)
    distances, indices = index.search(query_embedding, top_k)
    # Filter out any potential out-of-bound indices
    return [texts[i] for i in indices[0] if i < len(texts)]

# ---------------------------
# Step 5: Generation Using Retrieved Passages (RAG)
# ---------------------------
# Here we use T5 (t5-large) as our generation model.
# The retrieved passages are concatenated with the question as context.
generator = pipeline("text2text-generation", model="t5-large", tokenizer="t5-large")

def generate_answer(query, retrieved_texts):
    context = " ".join(retrieved_texts)
    # Prepare a prompt that includes both the question and the context.
    prompt = f"question: {query}  context: {context}"
    answer = generator(prompt, max_length=256, do_sample=False)
    return answer[0]['generated_text']

# ---------------------------
# Combine Everything into a RAG Pipeline
# ---------------------------
def run_rag(url, query, top_k=3):
    # Retrieve website content
    texts = scrape_website(url)
    if not texts:
        return "No content was scraped from the URL.", []

    # Compute embeddings and build FAISS index
    embeddings = embed_text(texts)
    index = build_faiss_index(embeddings)

    # Retrieve the most relevant passages
    retrieved_texts = search_faiss(index, query, texts, top_k=top_k)

    # Generate an answer using the retrieved context
    answer = generate_answer(query, retrieved_texts)
    return answer, retrieved_texts

# ---------------------------
# Example Usage
# ---------------------------
if __name__ == "__main__":
    url = "https://www.espncricinfo.com/series/wpl-2024-25-1463356/gujarat-giants-women-vs-mumbai-indians-women-5th-match-1469302/full-scorecard"
    query = "What was the score of Mumbia Indians"

    answer, retrieved_docs = run_rag(url, query)
    print("Generated Answer:", answer)
    print("\nRetrieved Passages:")
    for idx, passage in enumerate(retrieved_docs, 1):
        print(f"{idx}. {passage}\n")


Device set to use cpu


Generated Answer: MI Women won by 5 wickets (with 23 balls remaining) Zimbabwe won by 9 wickets (with 63 balls remaining)

Retrieved Passages:
1. MI Women won by 5 wickets (with 23 balls remaining)

2. Zimbabwe won by 9 wickets (with 63 balls remaining)

3. Mumbai started the tournament with a close loss against Delhi Capitals and will be itching to open their account

